# Owl2vec4OA
This process implements reading ontology files and transforming them into embeddings

In [ ]:
import owl2vec4mappings

# Parameters:
# ontology_file1
# ontology_file2
# mapping_file
# config_file
# uri_doc
# lit_doc
# mix_doc
# but there are some parameters you can change in config file
gensim_model = owl2vec4mappings.extract_owl2vec_model(ontology_file1 = "./case_studies/Data/snomed.body.owl",
                                                ontology_file2 = './case_studies/Data/fma.body.owl',
                                                mapping_file = './case_studies/Data/Trainval_U_sn2fmb.tsv',
                                                config_file = "./default1.cfg",
                                                uri_doc = True, lit_doc = True, mix_doc = True)

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sev_s\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO: 1
INFO: 
 Access the ontology ...
INFO: There are 148525 triples in the ontology
* Owlready2 * Creating new ontology fma.body <./case_studies/Data/fma.body.owl#>.
* Owlready2 * ADD TRIPLE ./case_studies/Data/fma.body.owl http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology
* Owlready2 *     ...loading ontology fma.body from ./case_studies/Data/fma.body.owl...
* OwlReady2 * Importing 930321 object triples from ontology ./case_studies/Data/fma.body.owl# ...
* OwlReady2 * Importing 841800 data triples from ontology ./case_studies/Data/fma.body.owl# ...
* Owlready2 * WARNING: DataProperty http://purl.org/sig/ont/fma/has_direct_shape_type belongs to more than one enti

In [ ]:
# Run
import configparser
config = configparser.ConfigParser()
config.read("default1.cfg")
output_folder=config['DOCUMENT']['cache_dir']

In [ ]:
#Gensim format
#Run
gensim_model.save(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings")

In [ ]:
#Txt format
#Run
gensim_model.wv.save_word2vec_format(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings.txt", binary=False)

In [ ]:
# Run
from gensim.models import Word2Vec
model = Word2Vec.load(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings")

In [ ]:
# Run
ranking_results = []
# save the scored candidate mappings in the same format as the original test.cands.tsv
f = open("scored.test.cands.tsv", "w")
f.write("SrcEntity" +'\t'+ "TgtEntity" +'\t'+ "TgtCandidates\n")
file = open(r"C:\Users\sev_s\Documents\OWL2Vec-Star\case_studies\Data\test.cands_sn2fb.tsv")

for i in file.readlines()[1:]:
    src_ref_class = i.split('\t')[0]
    tgt_ref_class = i.split('\t')[1]
    tgt_cands = i.split('\t')[2]
    # print(src_ref_class, tgt_ref_class, tgt_cands)
    tgt_cands = eval(tgt_cands)  # transform string into list or sequence
    scored_cands = []
    for tgt_cand in tgt_cands:
        # assign a score to each candidate with an OM system
        try:
            matching_score = model.wv.similarity(src_ref_class, tgt_cand)
        except:
            matching_score = 0
        scored_cands.append([tgt_cand, matching_score])
    ranking_results.append([src_ref_class, tgt_ref_class, scored_cands])
    f.write(f'{src_ref_class}\t{tgt_ref_class}\t{scored_cands}\n')
print("done")

In [ ]:
# Run
import numpy as np
distance_results = []
# save the scored candidate mappings in the same format as the original test.cands.tsv
f = open("distance.test.cands.tsv", "w")
f.write("SrcEntity" +'\t'+ "TgtEntity" +'\t'+ "TgtCandidates\n")
file = open(r"C:\Users\sev_s\Documents\OWL2Vec-Star\case_studies\Data\omim2ordo_test.cands_sn2fb.tsv")

for i in file.readlines()[1:]:
    src_ref_class = i.split('\t')[0]
    tgt_ref_class = i.split('\t')[1]
    tgt_cands = i.split('\t')[2]
    # print(src_ref_class, tgt_ref_class, tgt_cands)
    tgt_cands = eval(tgt_cands)  # transform string into list or sequence
    scored_cands = []
    for tgt_cand in tgt_cands:
        # assign a score to each candidate with an OM system
        try:
            euc_dist = np.linalg.norm(model.wv[src_ref_class] - model.wv[tgt_cand])
        except:
            euc_dist = np.inf
        scored_cands.append([tgt_cand, euc_dist])
    distance_results.append([src_ref_class, tgt_ref_class, scored_cands])
    f.write(f'{src_ref_class}\t{tgt_ref_class}\t{scored_cands}\n')
print("done")

In [ ]:
# Run
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# Load your Word2Vec model
model = Word2Vec.load(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings")

# Extract word vectors and corresponding words
words = list(model.wv.vocab.keys())
vectors = [model.wv[word] for word in words]

# Apply PCA to reduce dimensionality
num_dimensions = 2
pca = PCA(n_components=num_dimensions)
vectors_pca = pca.fit_transform(vectors)

# Elbow method for finding number of clusters

wcss = [] 
for i in range(1, 16): 
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(vectors_pca) 
    wcss.append(kmeans.inertia_)

plt.plot(range(1,16), wcss)

In [ ]:
# Apply KMeans clustering
num_clusters = 5  # You can change this based on your preference
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(vectors_pca)

# Visualize the clusters
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
for i in range(num_clusters):
    cluster_points = np.array([vectors_pca[j] for j in range(len(words)) if kmeans.labels_[j] == i])
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=colors[i], label=f'Cluster {i + 1}')

plt.title('Train-Val+ AML union LOgmap')
plt.legend()
plt.show()

In [ ]:
import random

# Choose a random index within the range of the words list
random_index = random.sample(range(len(words)), 30)

vector = {
    'X':vectors_pca[:,0],
    'Y':vectors_pca[:,1],
    'words': np.array(words),
    'label':kmeans.labels_
}

sample_vectors_pca = np.array([vector['X'][random_index], vector['Y'][random_index]]).T
sample_words = np.array(vector['words'][random_index])
sample_cluster = np.array(vector['label'][random_index])

# Visualize the clusters
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
markers = ['o', 'v', '^', '<', '>', 's', 'p', 'h', 'x', '+', 'd', '|', '_']

for i in range(num_clusters):
    cluster_points = np.array([sample_vectors_pca[j] for j in range(len(sample_words)) if sample_cluster[j] == i])
    plt.scatter(cluster_points[:,0], cluster_points[:,1], c=colors[i], marker=markers[i])

for i, txt in enumerate(sample_words):
    plt.annotate(txt, (sample_vectors_pca[i,0], sample_vectors_pca[i,1]), fontsize=5)